# NB1: Home-storage systems and operational strategies

*Distributed energy resources* are advocated to substitute big centralized power stations in an attempt to push the decarbonization of our energy systems. With the latest advanced of renewable energy integration, particularly rooftop PV, the next logical step is to combine these with energy storage systems and improve the local consumption of the locally generated electricity.

From the consumer perspective there might be also other advantages of owning an energy storage system beyond simply an altruistic goal of maximizing autarky and a low carbon footprint. Increasing electricity prices, fading feed-in tariffs and falling ownership costs of battery energy storage systems have created the ideal conditions for the economic benefit. This has lead to a recent significant growth of the home-storage market.
[It is estimated, that in Germany alone](https://arxiv.org/abs/2203.06762v2) 145,000 new home-storage systems with a combined capacity of 1.27 GWh were commissioned in 2021, increasing the total installed capacity to 3.5 GWh.

These batteries allow great amount of flexibility in their control, and for this reason designing proper operational strategies is an important aspect in the effort to get the most out of the investment.

This notebook explores the following:
- Time-series simulation of battery energy storage systems.
- Techno-economic analysis of home-storage systems.
- The impact of operational strategies on the system performance.

In [ ]:
import pandas as pd

In [ ]:
# plotting options
pd.options.plotting.backend = "plotly"
template = "plotly_white"
# template = "plotly_dark"

## Use case - dataset and base-scenario definition
For our analysis we take a look into a *representative* household with a solar PV system. The data is taken from the [*Standard Battery Application Profiles (SBAP) paper*](https://doi.org/10.1016/j.est.2019.101077), which provides curated profiles of multiple battery storage applications that researches can take to test against.

In [ ]:
profile = pd.read_csv("../data/household_profile.csv", index_col=0, parse_dates=True)
profile.plot(template=template, labels={"value": "Power [kW]"})

Let's take first a look into our baseline scenario: a system with no battery energy storage:

<div class="alert alert-block alert-info">
<b>Task!</b> Analyze the consumption and generation profiles:
<ul>
    <li> Calculate the residual load, include it as new column of the dataframe and plot it. </li>
    <li> Find the total energy consumption and generation in a year and calculate the resulting electricity costs. Consider 0.39 €/kWh for the electricity price and 0.07 €/kWh for the feed-in tariff. </li>
    <li> Implement functions to compute both the self-consumption-rate and the self-sufficiency-rate, use them to determine the values for this scenario. </li> 
</ul>
</div>

<div class="alert alert-block alert-warning">
<b>Hint!</b> Take attention when converting between power and energy. Don't forget to properly label the plot axes.
</div>

In [ ]:
# task
profile["residual"] = profile["load"] - profile["pv"]
profile["residual"].plot(template=template, labels={"value": "Power [kW]"}) #.update_yaxes(autorange="reversed")

In [ ]:
# task
def electricity_costs(grid_profile, electricity_price, feedin_tariff):
    grid_sell = grid_profile.loc[grid_profile < 0].sum() * 0.25 * feedin_tariff * -1  # negative sign
    grid_buy  = grid_profile.loc[grid_profile > 0].sum() * 0.25 * electricity_price
    return grid_buy - grid_sell

In [ ]:
# task
def self_consumption(grid_profile, pv_profile):
    feedin = grid_profile[grid_profile < 0].sum() * -1
    pv = pv_profile.sum()
    return 1 - (feedin / pv)

In [ ]:
# task
def self_sufficiency(grid_profile, load_profile):
    gridbuy = grid_profile[grid_profile > 0].sum()
    load = load_profile.sum()
    return 1 - (gridbuy / load) # energy/power?

In [ ]:
# task

# demand/consumption
total_demand = profile["pv"].sum() * 0.25
total_generation = profile["load"].sum() * 0.25
print(f"Total demand:      {total_demand:.0f} kWh")
print(f"Total generation:  {total_generation:.0f} kWh")

# electricity costs
electricity_price = 0.39
feedin_tariff = 0.07
costs_base = electricity_costs(profile["residual"], electricity_price, feedin_tariff)
print(f"Electricity costs: {costs_base:.2f} €")

# self-consumption / self-sufficiency
ssr_base = self_sufficiency(profile["residual"], profile["load"])
scr_base = self_consumption(profile["residual"], profile["pv"])
print(f"Self-sufficiency:   {ssr_base*100:.2f} %")
print(f"Self-consumption:   {scr_base*100:.2f} %")

## Greedy strategy

To make better use of the solar energy, the household decides to install a storage system, designed with 5 kWh nominal capacity and 5 kW max. power. Nonetheless, the battery won't do the work for itself... We have to tell it first what to do!

For our first attempt, we can try out the most trivial solutions possible to **improve the self-consumption**: make the storage system observe the residual load and as soon as the PV generation exceeds the current load, it will charge the battery. Then, as soon when the PV generation falls below the demand, the battery will discharge back to cover the difference. This *dumb* strategy tries to blindly storage all the solar energy available, that is why it is given the name of *greedy* strategy.

<div class="alert alert-block alert-info">
<b>Task!</b> Analyze the greedy strategy:
<ul>
    <li> Implement the greedy strategy for self-consumption increase. Define a function that takes the profile dataframe and the storage system size (capacity and power), and returns a new dataframe with the battery SOC and power, and the resulting grid power at each timestep. Check during the entire operation, that the power and SOC limits are not violated. Consider the system losses with charge/discharge efficiencies of 95%. </li>
    <li> Calculate the new self-consumption and self-sufficiency rates. Evaluate the improvement from the base scenario. </li>
    <li> Calculate the new electricity bill and the resulting <i>savings</t>. </li>
</ul>
</div>

<div class="alert alert-block alert-warning">
<b>Hint!</b> Iterate through the dataframe to calculate the power and SOC at each timestep. Follow the convention of positive power for charging, negative power for discharging.
Again, take attention when converting between power and energy values.
</div>

In [ ]:
def greedy_strategy(profile, capacity, max_power, eff=0.95, initial_soc=0.0):
    df = profile.copy() # make a new copy of the dataframe
    # add new empty columns to the dataframe
    df["grid"] = 0.0   # grid power in kW
    df["power"] = 0.0  # battery power in kW
    df["soc"]  = 0.0   # battery SOC in p.u.

    # task
    dt = 0.25
    soc =  initial_soc

    for time, residual in profile["residual"].iteritems():
        if residual < 0:
            # charge
            power = min(abs(residual), max_power)
            soc_new = min(soc + (power * dt * eff) / capacity, 1.0) 
        else:
            # discharge
            power = min(abs(residual), max_power)
            soc_new = max(soc - (power * dt * (1/eff)) / capacity, 0.0) 
        
        # battery power, soc state and power exchange with the grid
        power_real = -(soc_new - soc) / dt * capacity # positive -> charge / negative -> discharge
        grid = residual - power_real
        soc = soc_new

        df.loc[time, "grid"] = grid
        df.loc[time, "power"] = power_real
        df.loc[time, "soc"]  = soc
    
    return df

Let's simulate the operation and visualize the results.

In [ ]:
capacity = 5.0 # kWh
max_power = 5.0 # kW

In [ ]:
df_greedy = greedy_strategy(profile=profile, capacity=capacity, max_power=max_power) # this can take some time to compute

In [ ]:
df_greedy[["residual", "power"]].plot(template=template, labels={"value": "Power [kW]"})# .update_yaxes(autorange="reversed")

In [ ]:
df_greedy["soc"].plot(template=template, labels={"value": "SOC"})

In [ ]:
# task

# electricity costs
electricity_price = 0.39
feedin_tariff = 0.07
costs_greedy = electricity_costs(df_greedy["grid"], electricity_price, feedin_tariff)
print(f"Electricity costs: {costs_greedy:.2f} € ({costs_greedy-costs_base:.2f} €, {(costs_greedy-costs_base)/costs_base*100:-.2f} %)")

# self-consumption / self-sufficiency
ssr_greedy = self_sufficiency(df_greedy["grid"], df_greedy["load"])
scr_greedy = self_consumption(df_greedy["grid"], df_greedy["pv"])
print(f"Self-sufficiency:   {ssr_greedy*100:.2f} % ({(ssr_greedy - ssr_base)/ssr_base*100:+.2f}%)")
print(f"Self-consumption:   {scr_greedy*100:.2f} % ({(scr_greedy - scr_base)/scr_base*100:+.2f}%)")

## Feed-in damp strategy

The *greedy* strategy seems to get the work done. After all, it is the *go-to* strategy most vendors probably embed for their home-storage systems. Nonetheless, there might be room for improvement.

The battery seems to full up very early, specially on summer days. This means the battery is often unable to mitigate the peak generation of the PV, which impact may cause the need of expensive reinforcements of the grid infrastructure. With that in mind, [Zeh & Witzman](https://doi.org/10.1016/j.egypro.2014.01.164) propose the *feed-in damp* strategy. This follows a similar logic to the *greedy* tactic: check the residual load and charge and discharge whenever available. With a twist however, the *feed-in damp* strategy aims to fill the storage 1 hour before sunset and limits the power based on the remaining capacity and time.

$$ P_{max, damp} = \frac{E_{remaining}}{t_{sunset} - t_{now} - 1 \text{ h}} $$

Can we benefit both our household and the grid? Let's see how this algorithm performs.

The challenging element of this method is having to estimate the time of the sunset for a particular day (and location!). Luckily for *any* problem we might encounter, someone else might have implemented it already and published as a Python package. This time the [*suntime*](https://github.com/SatAgro/suntime) package comes to our rescue and saves us for having to worry about the details.

In [ ]:
# documentation: https://github.com/SatAgro/suntime
from suntime import Sun
from datetime import timedelta, timezone

Here is a small demo on how to use *suntime*

In [ ]:
# calculate sunset time for a particular location and date
latitude = 48.8
longitude = 11.34
sun = Sun(latitude, longitude) # Munich location

# suntime returns time in UTC, so it has to be first translated to the local timezone
sunset_time_utc   = sun.get_sunset_time()
sunset_time_local = sunset_time_utc.astimezone(timezone(timedelta(hours=+1), name="Europe/Berlin")) # "Europe/Berlin" timezone
print(f"UTC time:   ", sunset_time_utc)
print(f"Local time: ", sunset_time_local)

In [ ]:
# calculate remainig time for the sunset
day = profile.index[1]
sunset_time = sun.get_sunset_time(day).replace(tzinfo=None) # delete timezone metadata to allow datetime-arithmetic
offset = sunset_time - day
print(offset)
print(offset.seconds, "s") # offset in seconds

<div class="alert alert-block alert-info">
<b>Task!</b> Analyze the feed-in damp strategy:
<ul>
    <li> Implement the feed-in damp strategy for self-consumption increase. Define a function that takes the profile dataframe and the storage system size (capacity and power), and returns a new dataframe with the SOC and power of the battery at each timestep. Check during the entire operation, that the power and SOC limits are not violated. Consider the system losses with charge/discharge efficiencies of 95%. </li>
    <li> Calculate the new self-consumption and self-sufficiency rates. Evaluate the improvement from the base scenario. </li>
    <li> Calculate the new electricity bill and the resulting <i>savings</t>. </li>
</ul>
</div>

<div class="alert alert-block alert-warning">
<b>Hint!</b> Use <i> suntime </i> to estimate the sunset time for each day.
</div>

In [ ]:
def feedin_damp(profile, capacity, max_power, eff=0.95, initial_soc=0.0):
    df = profile.copy() # make a new copy of the dataframe
    # add new empty columns to the dataframe
    df["grid"] = 0.0   # grid power in kW
    df["power"] = 0.0  # battery power in kW
    df["soc"]  = 0.0   # battery SOC in p.u.
    
    # task
    dt = 0.25
    soc = initial_soc

    latitude, longitude = (48.8, 11.34) # Munich location
    sun = Sun(latitude, longitude)

    for time, residual in df["residual"].iteritems():
        if residual < 0:
            # charge
            sunset_time = sun.get_sunset_time(time)  # time in UTC
            sunset_time = sunset_time.astimezone(timezone(timedelta(hours=1.0))).replace(tzinfo=None) # local time (Germany)
            target_time = sunset_time - timedelta(hours=1.0)

            remaining_time  = target_time - time
            remaining_hours = remaining_time.seconds / 3600
            
            energy_remaining = capacity * (1 - soc)
            
            if remaining_hours > 0:  # Preventing division by 0
                power_ref = energy_remaining / (remaining_hours)  # Wh / h
            else:
                power_ref = energy_remaining / 1 # -> last hour
            
            power_ref = min(power_ref, max_power)
        
            power = min(abs(residual), power_ref)
            soc_new = min(soc + (power * dt * eff) / capacity, 1.0)
        else:
            # discharge
            power = min(abs(residual), max_power)
            soc_new = max(soc - (power * dt * (1/eff)) / capacity, 0.0)     
        
        # battery power, soc state and power exchange with the grid
        power_real = -(soc_new - soc) / dt * capacity # negative -> charge / positive -> discharge
        grid = residual - power_real
        soc = soc_new

        df.loc[time, "grid"] = grid
        df.loc[time, "power"] = power_real
        df.loc[time, "soc"]  = soc

    return df

In [ ]:
df_damp = feedin_damp(profile, capacity=capacity, max_power=max_power) # this will also take some time to compute

In [ ]:
df_damp[["residual", "grid", "power"]].plot(template=template)

In [ ]:
# task

# electricity costs
electricity_price = 0.39
feedin_tariff = 0.07
costs_damp = electricity_costs(df_damp["grid"], electricity_price, feedin_tariff)
print(f"Electricity costs: {costs_damp:.2f} € ({costs_damp-costs_base:.2f} €, {(costs_damp-costs_base)/costs_base*100:-.2f} %)")

# self-consumption / self-sufficiency
ssr_damp = self_sufficiency(df_damp["grid"], df_damp["load"])
scr_damp = self_consumption(df_damp["grid"], df_damp["pv"])
print(f"Self-sufficiency:   {ssr_damp*100:.2f} % ({(ssr_damp - ssr_base)/ssr_base*100:+.2f}%)")
print(f"Self-consumption:   {scr_damp*100:.2f} % ({(scr_damp - scr_base)/scr_base*100:+.2f}%)")

### Comparing *Greedy* vs. *Feed-in Damp*

We have implemented and analyzed both operational strategies for the self-consumption increase. 
There is a lot going on so let's compare both approaches hand-in-hand with some visualizations.

In [ ]:
dfbat = pd.concat([df_greedy["power"].rename("power - greedy"), df_damp["power"].rename("power - feed-in damp")], axis=1)
dfbat.plot(template=template, labels={"value": "Power [kW]"})

In [ ]:
dfbat.plot.hist(template=template, log_y=True, labels={"value": "Power [kW]"}).update_layout(barmode='overlay').update_traces(opacity=0.75)

In [ ]:
dfsoc = pd.concat([df_greedy["soc"].rename("soc - greedy"), df_damp["soc"].rename("soc - feed-in damp")], axis=1)
dfsoc.plot(template=template, labels={"value": "SOC"})

In [ ]:
dfsoc.plot.hist(template=template, log_y=True, labels={"value": "SOC"}).update_layout(barmode='overlay').update_traces(opacity=0.75)

In [ ]:
dfgrid = pd.concat([df_greedy["grid"].rename("soc - greedy"), df_damp["grid"].rename("soc - feed-in damp")], axis=1)
dfgrid.plot(template=template, labels={"value": "Power [kW]"})

In [ ]:
dfgrid.plot.hist(template=template, log_y=True, labels={"value": "Grid power [kW]"}).update_layout(barmode='overlay').update_traces(opacity=0.75)

<div class="alert alert-block alert-info">
<b> Task! </b> Compare both strategies. List pros and cons, assessing from both the homeowner and the grid operator perspective.
</div>

<div class="alert alert-block alert-warning">
<b>Hint!</b> Write your response in a <b> markdown</b> cell.
</div>

**Greedy:**
- *Pros:*
    * Simple rule-based implementation.
    * Achieves the highest self-sufficiency/self-consumption and resultant cost savings.
- *Cons:*
    * High feed-in injection and high power gradients have a negative impact in the grid operations.
    * Long periods with high SOC and high power rates increase battery degradation.


**Feed-in damp:**
- *Pros:*
    * Feed-in peaks are reduced, alleviating the grid and potentially avoiding grid-reinforcements and enabling the integrating more home-solar-systems.
    * If feed-in limits are introduces, the feed-in damp reduces the need of curtailing the PV generation.
    * Lower power rates and more balanced SOC prolongs battery lifetime.
- *Cons:*
    * More complex implementation, *forecast* required.
    * Susceptible to *prediction errors*, achieves a slightly lower self-sufficiency and cost savings for the homeowner.